In [10]:
import sys
from pathlib import Path
import espnet
from espnet2.bin.asr_inference_streaming import Speech2TextStreaming
from espnet_model_zoo.downloader import ModelDownloader
import numpy as np
import wave
import yaml
import time
from datetime import datetime
import pyaudio
from typing import Optional, List, Tuple, Dict
from dataclasses import dataclass

In [11]:
@dataclass
class SPLMeasurement:
    """SPLの測定結果を保持するクラス"""
    last_word: str
    last_word_time: float  # 音声内での最後の単語の時間
    transcription_time: float  # 認識完了時間
    spl: float  # Speech Perceived Latency

class StreamingSpeechRecognizer:
    """ストリーミング音声認識器のクラス"""
    def __init__(self, config_path: str, model_path: str):
        # ESPnetの設定
        with open(config_path, 'r') as f:
            config = yaml.safe_load(f)
            
        # Speech2Textの初期化
        self.speech2text = Speech2TextStreaming(
            asr_train_config=config_path,
            asr_model_file=model_path,
            token_type=None,
            bpemodel=None,
            maxlenratio=0.0,
            minlenratio=0.0,
            beam_size=20,
            ctc_weight=0.5,
            lm_weight=0.0,
            penalty=0.0,
            nbest=1,
            device="cpu",
            disable_repetition_detection=True,
            decoder_text_length_limit=0,
            encoded_feat_length_limit=0
        )
        
        self.chunk_size = 640  # 16kHz * 0.04s = 640 samples per chunk

    def measure_spl(self, wav_path: str, verbose: bool = True) -> SPLMeasurement:
        """SPLを測定する関数"""
        # 音声データの読み込み
        with wave.open(wav_path, 'rb') as wav_file:
            rate = wav_file.getframerate()
            nframes = wav_file.getnframes()
            audio_data = wav_file.readframes(nframes)
            
        # 音声データの変換
        speech = np.frombuffer(audio_data, dtype='int16').astype(np.float16) / 32767.0
        
        start_time = time.time()
        last_word = ""
        last_word_time = 0.0
        current_text = ""
        
        if verbose:
            print("\nSPL Measurement Progress:")
            print("Time | Current Text | Last Word")
            print("-" * 50)
        
        # チャンクごとに処理
        for i in range(0, len(speech), self.chunk_size):
            chunk = speech[i:i + self.chunk_size]
            current_time = i / rate
            
            # 最後のチャンクかどうか
            is_final = (i + self.chunk_size >= len(speech))
            
            # 音声認識
            results = self.speech2text(
                speech=chunk,
                is_final=is_final
            )
            
            if results and len(results) > 0:
                text = results[0][0]  # 最良の認識結果を取得
                
                if text and text != current_text:
                    current_text = text
                    words = text.strip().split()
                    if words:
                        last_word = words[-1]
                        last_word_time = current_time
                        
                        if verbose:
                            print(f"{current_time:.2f}s | {text} | {last_word}")
        
        end_time = time.time()
        transcription_time = end_time - start_time
        spl = (transcription_time * 1000) - (last_word_time * 1000)  # ミリ秒に変換
        
        measurement = SPLMeasurement(
            last_word=last_word,
            last_word_time=last_word_time,
            transcription_time=transcription_time,
            spl=spl
        )
        
        if verbose:
            print("\nSPL Measurement Results:")
            print(f"Last Word: {measurement.last_word}")
            print(f"Last Word Time: {measurement.last_word_time:.2f}s")
            print(f"Transcription Time: {measurement.transcription_time:.2f}s")
            print(f"SPL: {measurement.spl:.2f}ms")
            
        return measurement

    def measure_realtime_spl(self, duration: int = 5, verbose: bool = True) -> SPLMeasurement:
        """リアルタイムでSPLを測定する関数"""
        CHUNK = 2048
        FORMAT = pyaudio.paInt16
        CHANNELS = 1
        RATE = 16000
        
        p = pyaudio.PyAudio()
        stream = p.open(
            format=FORMAT,
            channels=CHANNELS,
            rate=RATE,
            input=True,
            frames_per_buffer=CHUNK
        )
        
        start_time = time.time()
        last_word = ""
        last_word_time = 0.0
        current_text = ""
        
        if verbose:
            print("\nRealtime SPL Measurement:")
            print("Time | Current Text | Last Word")
            print("-" * 50)
        
        try:
            for i in range(0, int(RATE/CHUNK * duration)):
                current_time = i * CHUNK / RATE
                
                # 音声データの読み取り
                data = stream.read(CHUNK)
                audio_data = np.frombuffer(data, dtype='int16')
                audio_data = audio_data.astype(np.float16) / 32767.0
                
                # 最後のチャンクかどうか
                is_final = (i == int(RATE/CHUNK * duration) - 1)
                
                # 音声認識
                results = self.speech2text(
                    speech=audio_data,
                    is_final=is_final
                )
                
                if results and len(results) > 0:
                    text = results[0][0]
                    
                    if text and text != current_text:
                        current_text = text
                        words = text.strip().split()
                        if words:
                            last_word = words[-1]
                            last_word_time = current_time
                            
                            if verbose:
                                print(f"{current_time:.2f}s | {text} | {last_word}")
                    
        finally:
            stream.stop_stream()
            stream.close()
            p.terminate()
        
        end_time = time.time()
        transcription_time = end_time - start_time
        spl = (transcription_time * 1000) - (last_word_time * 1000)
        
        measurement = SPLMeasurement(
            last_word=last_word,
            last_word_time=last_word_time,
            transcription_time=transcription_time,
            spl=spl
        )
        
        if verbose:
            print("\nRealtime SPL Measurement Results:")
            print(f"Last Word: {measurement.last_word}")
            print(f"Last Word Time: {measurement.last_word_time:.2f}s")
            print(f"Transcription Time: {measurement.transcription_time:.2f}s")
            print(f"SPL: {measurement.spl:.2f}ms")
            
        return measurement

In [12]:
def main():
    """メイン関数"""
    # パスの設定
    data_dir = Path("data")
    config_path = str(data_dir / "config.yaml")
    model_path = str(data_dir / "valid.acc.best.pth")
    
    # 認識器の初期化
    recognizer = StreamingSpeechRecognizer(config_path, model_path)
    
    # ファイルからのSPL測定
    wav_path = "sample.wav"
    print(f"\nMeasuring SPL for file: {wav_path}")
    file_spl = recognizer.measure_spl(wav_path)
    
    # # リアルタイムSPL測定
    # print("\nMeasuring realtime SPL (5 seconds)...")
    # realtime_spl = recognizer.measure_realtime_spl(duration=5)

if __name__ == "__main__":
    main()


Measuring SPL for file: sample.wav

SPL Measurement Progress:
Time | Current Text | Last Word
--------------------------------------------------
1.88s | <dysfl> す ー </dysfl> あ の | の
2.40s | <dysfl> い </dysfl> <dysfl> あ の </dysfl> す い ま せ ん ふ ら | ら
2.88s | <dysfl> す ー </dysfl> <dysfl> あ の </dysfl> す い ま せ ん ふ ら ん す 料 | 料
3.40s | <dysfl> い す ー </dysfl> <dysfl> あ の </dysfl> す い ま せ ん ふ ら ん す 料 り あ を と | と
3.92s | <dysfl> す ー </dysfl> <dysfl> あ の </dysfl> す い ま せ ん ふ ら ん す 料 り あ を 捉 え で 探 | 探
4.44s | <dysfl> す ー </dysfl> <dysfl> あ の </dysfl> す い ま せ ん ふ ら ん す 料 り あ を 捉 え で 探 し て い る | る
4.96s | <dysfl> す ー </dysfl> <dysfl> あ の </dysfl> す い ま せ ん ふ ら ん す 料 り あ を 捉 え で 探 し て い る ん で す け ど | ど
5.96s | <dysfl> い す ー </dysfl> <dysfl> あ の </dysfl> す い ま せ ん ふ ら ん す 料 り あ を 捉 え で 探 し て い る ん で す け ど ー <dysfl> え ー | ー
6.48s | <dysfl> す ー </dysfl> <dysfl> あ の </dysfl> す い ま せ ん ふ ら ん す 料 り あ を 捉 え で 探 し て い る ん で す け ど ー <dysfl> え っ と ー </dysfl> そ | そ
7.00s | <dysfl> す ー </dysfl> <dysfl> あ の </dys